In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/FirstCompetition/
%ls

/content/drive/MyDrive/FirstCompetition
'3) Convolutional Neural Network (Skeleton).ipynb'
 BalanceDataset.ipynb
 balanced_dataset/
'Copy of Giorno2.ipynb'
'Copy of transfer_learning.ipynb'
 data_augmentation_experiments/
 data-augmentation_tuning.ipynb
 dataset/
 dataset.zip
 Erica_Attempt.ipynb
 EricaTest/
 EricaTest2/
 Erica_Xception.ipynb
 Erica_Xception_tentativo3.ipynb
 Erica_Xception_tuner_copied_by_Gianni.ipynb
 FirstSubmission/
 FisrtAttempt.ipynb
 Gianni_attempt.ipynb
 Gianni_transfer_tuner.ipynb
 Giorno2.ipynb
 Luca_attempt.ipynb
'MNV3 (1).ipynb'
 model.png
 requirements.txt
 results_data_augmentation.ipynb
 test1/
 test2/
 test3/
 test4/
 test_vgg/
 ToyNotebook.ipynb
 ToySubmission/
 ToySubmission.zip
 transfer_learning.ipynb
 TransferLearningModel/
 Untitled0.ipynb


In [3]:
!rm -r /dataset
!rm -r /balanced_dataset

rm: cannot remove '/dataset': No such file or directory
rm: cannot remove '/balanced_dataset': No such file or directory


In [4]:
!mkdir /dataset
!mkdir /balanced_dataset

In [5]:
!unzip dataset -d /dataset
!unzip dataset -d /balanced_dataset

Output streaming troncato alle ultime 5000 righe.
  inflating: /balanced_dataset/training/Tomato/37979.jpg  
  inflating: /balanced_dataset/training/Tomato/37980.jpg  
  inflating: /balanced_dataset/training/Tomato/37982.jpg  
  inflating: /balanced_dataset/training/Tomato/37983.jpg  
  inflating: /balanced_dataset/training/Tomato/37984.jpg  
  inflating: /balanced_dataset/training/Tomato/37985.jpg  
  inflating: /balanced_dataset/training/Tomato/37988.jpg  
  inflating: /balanced_dataset/training/Tomato/37989.jpg  
  inflating: /balanced_dataset/training/Tomato/37990.jpg  
  inflating: /balanced_dataset/training/Tomato/37992.jpg  
  inflating: /balanced_dataset/training/Tomato/37993.jpg  
  inflating: /balanced_dataset/training/Tomato/37994.jpg  
  inflating: /balanced_dataset/training/Tomato/37995.jpg  
  inflating: /balanced_dataset/training/Tomato/37996.jpg  
  inflating: /balanced_dataset/training/Tomato/37997.jpg  
  inflating: /balanced_dataset/training/Tomato/37998.jpg  
  infl

In [7]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import shutil

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tqdm import tqdm


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tfk.layers

In [8]:
# Random seed for reproducibility
seed = 131

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [9]:
ORI_DS = '/dataset/training/'

DEST_DS = '/balanced_dataset/training/'

IMAGE_SIZE = (256,256)
CHANNELS = 3
INPUT_SHAPE = (IMAGE_SIZE)+(CHANNELS,)

BATCH_SIZE = 128
EPOCHS = 500

labels = ['Apple',        # 0
          'Blueberry',    # 1
          'Cherry',       # 2
          'Corn',         # 3
          'Grape',        # 4
          'Orange',       # 5
          'Peach',        # 6
          'Pepper',       # 7
          'Potato',       # 8
          'Raspberry',    # 9
          'Soybean',      # 10
          'Squash',       # 11
          'Strawberry',   # 12
          'Tomato'        # 13
]

In [10]:
images = {}
bal_images = {}

for label in labels:
  images[label] = glob(ORI_DS + label + '/*')
  bal_images[label] = glob(DEST_DS + label + '/*')
  print(label,len(images[label]),len(bal_images[label]))

max = np.max([len(images[label]) for label in labels])
print('max:',max)

Apple 988 988
Blueberry 467 467
Cherry 583 583
Corn 1206 1206
Grape 1458 1458
Orange 1748 1748
Peach 977 977
Pepper 765 765
Potato 716 716
Raspberry 264 264
Soybean 1616 1616
Squash 574 574
Strawberry 673 673
Tomato 5693 5693
max: 5693


In [11]:
def get_generator_for_class(label,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            vertical_flip=True,
                            rotation_range=45,
                            fill_mode='reflect',
                            seed=seed):

  datagen = ImageDataGenerator(
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        fill_mode=fill_mode,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip,
        vertical_flip=vertical_flip,
        rotation_range=rotation_range)

  class_generator = datagen.flow_from_directory(
      directory=ORI_DS,
      save_to_dir=DEST_DS+label,
      save_prefix='aug',
      save_format='jpg',
      classes=[label],
      target_size=(256,256),
      color_mode = 'rgb',
      batch_size=1,
      class_mode=None,
      seed = seed)
  
  return class_generator

In [12]:
ori_images = {}
bal_images = {}

for label in labels:
  ori_images[label] = glob(ORI_DS + label + '/*')
  bal_images[label] = glob(DEST_DS + label + '/*')

  ori_len = len(ori_images[label])
  bal_len = len(bal_images[label])

  if bal_len < max:

    print('beginning augmentation for images of class: ' + label)

    print('initializing random image generator...')
    gen = get_generator_for_class(label)

    print('adding augmented images...')
    for _ in tqdm(range(max-ori_len)):
      gen.next()

  else:

    print('class ' + label + ' already balanced.')


beginning augmentation for images of class: Apple
initializing random image generator...
Found 988 images belonging to 1 classes.
adding augmented images...


100%|██████████| 4705/4705 [01:08<00:00, 68.24it/s]


beginning augmentation for images of class: Blueberry
initializing random image generator...
Found 467 images belonging to 1 classes.
adding augmented images...


100%|██████████| 5226/5226 [01:15<00:00, 69.20it/s]


beginning augmentation for images of class: Cherry
initializing random image generator...
Found 583 images belonging to 1 classes.
adding augmented images...


...

beginning augmentation for images of class: Squash
initializing random image generator...
Found 574 images belonging to 1 classes.
adding augmented images...


100%|██████████| 5119/5119 [01:16<00:00, 66.87it/s]


beginning augmentation for images of class: Strawberry
initializing random image generator...
Found 673 images belonging to 1 classes.
adding augmented images...


100%|██████████| 5020/5020 [01:14<00:00, 67.39it/s]

class Tomato already balanced.


In [13]:
!cp -r /balanced_dataset /content/drive/MyDrive/FirstCompetition/balanced_dataset

In [14]:
images = {}
bal_images = {}

for label in labels:
  images[label] = glob(ORI_DS + label + '/*')
  bal_images[label] = glob(DEST_DS + label + '/*')
  print(label,len(images[label]),len(bal_images[label]))

max = np.max([len(images[label]) for label in labels])
print('max:',max)

Apple 988 5693
Blueberry 467 5693
Cherry 583 5693
Corn 1206 5693
Grape 1458 5693
Orange 1748 5693
Peach 977 5693
Pepper 765 5693
Potato 716 5693
Raspberry 264 5693
Soybean 1616 5693
Squash 574 5693
Strawberry 673 5693
Tomato 5693 5693
max: 5693
